# TFM MPVD Quique Ribera

## Creación de crónicas de partidos de fútbol a través de IA


En este proyecto se va a realizar un sitio web que recoja crónicas automatizadas de los partidos de fútbol de la categoría "2ª Regional Juvenil - Liga 13" organizada por la Federación de Fútbol de la Comunidad Valenciana.

La obtención de los datos se ha realizado con Python, a través del siguiente NoteBook, en el que se han utilizado las librerías: BeautifulSoup, operator, random, string, tqdm.

Para la publicación de estas, se ha utilizado el tema de Jekyll “Minima” que se puede encontrar en el repositorio de Github https://github.com/jekyll/minima.

Para la realización del TFM se ha contado con la ayuda de Martín Nadal y de Julián Pérez. Sin olvidar lo aprendido durante los módulos realizados con todos los profesores del Claustro del máster.



### Instalación e importación de librerías

Lo primero que hacemos es instalar e importar las librerías necesarias.

Para poder obtener los datos de los partidos de la web [FFCV](https://resultadosffcv.isquad.es/partido.php?id_partido=3679276&id_torneo=39882&jornada=30) utilizamos:
 - **_[Requests](https://requests.readthedocs.io/en/latest/)_ :** Esta librería la usamos para realizar las peticiones http.
 - **_[BeautifulSoup](https://beautiful-soup-4.readthedocs.io/en/latest/)_ :** Nos permitirá extraer los datos de las páginas web.
    
Para el tratamiento de estos datos utilizamos las librerías siguientes:

 - **_[operator](https://docs.python.org/3/library/operator.html)_ :** Esta librería permite realizar funciones de manera sencilla.
 - **_[random](https://docs.python.org/3/library/random.html)_ :** Gracias a Rondom podremos hacer que se seleccionen las partes de las crónicas de forma aleatoria haciendo más difícil que se generen con la misma estructura.
 - **_[string](https://docs.python.org/3/library/string.html)_ :** Esta librería será la que nos dé la posibilidad de incluir los datos extraídos dentro de las cadenas de texto mediante su método .format .
 - **_[tqdm](https://pypi.org/project/tqdm/)_ :** Esta librería genera una barra de progreso para darnos de manera visual el porcentaje de crónicas creadas.   
        
    

In [ ]:
# Instalación de las librerías

!pip install requests
!pip install bs4
!pip install operator
!pip install random
!pip install string
!pip install tqd

In [462]:
# Después de instalar debemos importar las librerías para poder usarlas 

import requests 
from bs4 import BeautifulSoup
import operator
import random
import string
from tqdm import tqdm

### Obtención de datos de los partidos

Creamos la variable partidoUrl con el enlace a un partido de la competición y la variable headers que permite que el cliente y el servidor pasen información con una solicitud o respuesta HTTP para poder obtener la información de la web a través del módulo request. Estas variables son las que se pasan en la variable siguiente r.

En la variable soup almacena mediante la librería BeautifulSoup los datos obtenidos en la variable r y convierte todos los datos de la web obtenidos en texto.

Las variables localSelector y visSelector almacenan las columnas de la web que contienen la información de los jugadores titulares, suplentes y técnicos de cada equipo.

Con estos datos se almacena los titulares en local_titulares y visitante_titulares , suplentes en local_suplentes y visitante_suplentes y técnicos en local_tecnicos y visitante_tecnicos.

A lo largo del Notebook podremos ver como algunas variables están creadas con el formato CamelCase, como localSelector o visSelector, mientras que otras variables tendrán los nombres separados con _ ,como local_tecnicos.

Con todo esto ya podemos parsear los datos de los jugadores titulares, suplentes y almacenarlos en diccionarios que contengan todos esos datos como el nombre, el dorsal, goles, si entran a jugar, si salen del campo sustituidos... También se realiza lo mismo con los entrenadores para conseguir sus nombres.

In [463]:
partidoUrl = 'https://resultadosffcv.isquad.es/partido.php?id_partido=3679276&id_torneo=39882&jornada=30'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
r = requests.get(partidoUrl,headers=headers)

soup = BeautifulSoup(r.text,'html.parser')
    
localSelector = 'div.col-lg-4:nth-child(1)'
visSelector   = 'div.col-lg-4:nth-child(3)'
    
(local_titulares,local_suplentes,local_tecnicos) = \
    [jugadores.select('li') \
        for jugadores in soup.select_one(localSelector).select('ul')]

(visitante_titulares,visitante_suplentes,visitante_tecnicos) = \
    [jugadores.select('li') \
        for jugadores in soup.select_one(visSelector).select('ul')]

def parseJugador(jugadorElem):
        
    etiquetas = [etiqueta.text.strip() for etiqueta in jugadorElem.select('span.span_etiqueta_alin')] 
    dorsal    = jugadorElem.select_one('span').text.strip()
    minutos   = [event.text for event in jugadorElem.select('span.minutos')]
    eventos   = [event['src'] for event in jugadorElem.select('div>img')]
    
    jugadorText = jugadorElem.text.strip()
    #quitamos los minutos directamente de la cadena
    for minuto in minutos:
        jugadorText = jugadorText.replace(minuto,'')

    #Para quedarnos con el texto 'padre' tenemos que quitar el texto de los hijos    
    jugadorElems = jugadorText.split(' ')
    for elem in etiquetas + [dorsal] :
        jugadorElems.remove(elem)
    nombre = ' '.join(jugadorElems).strip()
            
    entradas = [int(minutos[idx][:-1]) for idx in  [i for i,v in enumerate(eventos) if v.endswith('sustitucion_entrada_icono.png')]]
    salidas  = [int(minutos[idx][:-1]) for idx in  [i for i,v in enumerate(eventos) if v.endswith('sustitucion_salida_icono.png')]]
    goles    = [int(minutos[idx][:-1]) for idx in  [i for i,v in enumerate(eventos) if v.endswith('balon_icono.png')]]
    
    return {'nombre':nombre,'dorsal':int(dorsal),'etiquetas':etiquetas,'entradas':entradas,'salidas':salidas,'goles':goles}

def parseEntrenador(entrenadorElem):
    etiquetas_entrenador = [etiqueta.text for etiqueta in entrenadorElem.select('span.span_etiqueta_alin')]
    entrenadorElem = entrenadorElem.text.split(' ')
    
    nombre = ' '.join(entrenadorElem)
    for etiqueta in etiquetas_entrenador:
        nombre = nombre.replace(etiqueta,'')
    return {'nombre': nombre.strip()}

### Obtener los datos del partido

El siguiente paso es extraer la máxima información posible del partido como el número de la jornada, nombre del estadio, los nombres de los equipos, los datos de los jugadores y entrenadores parseados anteriormente, toda la información sobre los goles (cantidad, quién los mete, minuto de los goles, goles en cada parte...), la información de los cambios, resultado... y todo esto lo incluimos en un diccionario que almacenará la información y que utilizaremos sus elementos para las cadenas de texto que formarán las partes de las crónicas.

Durante la celda siguiente habrán explicaciones comentadas para ciertas partes del código.

In [464]:
def parsearPartido(partidoUrl):
    
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    r = requests.get(partidoUrl,headers=headers)
    soup = BeautifulSoup(r.text,'html.parser')
    
    localSelector = 'div.col-lg-4:nth-child(1)'
    visSelector   = 'div.col-lg-4:nth-child(3)'

    (local_titulares,local_suplentes,local_tecnicos) = \
        [jugadores.select('li') \
            for jugadores in soup.select_one(localSelector).select('ul')]

    (visitante_titulares,visitante_suplentes,visitante_tecnicos) = \
        [jugadores.select('li') \
            for jugadores in soup.select_one(visSelector).select('ul')]
    
    def parseJugador(jugadorElem):
        
        etiquetas = [etiqueta.text.strip() for etiqueta in jugadorElem.select('span.span_etiqueta_alin')] 
        dorsal    = jugadorElem.select_one('span').text.strip()
        minutos   = [event.text for event in jugadorElem.select('span.minutos')]
        eventos   = [event['src'] for event in jugadorElem.select('div>img')]

        jugadorText = jugadorElem.text.strip()
        #quitamos los minutos directamente de la cadena
        for minuto in minutos:
            jugadorText = jugadorText.replace(minuto,'')

        #Para quedarnos con el texto 'padre' tenemos que quitar el texto de los hijos    
        jugadorElems = jugadorText.split(' ')
        for elem in etiquetas + [dorsal] :
            jugadorElems.remove(elem)
        nombre = ' '.join(jugadorElems).strip()

        entradas = [int(minutos[idx][:-1]) for idx in  [i for i,v in enumerate(eventos) if v.endswith('sustitucion_entrada_icono.png')]]
        salidas  = [int(minutos[idx][:-1]) for idx in  [i for i,v in enumerate(eventos) if v.endswith('sustitucion_salida_icono.png')]]
        goles    = [int(minutos[idx][:-1]) for idx in  [i for i,v in enumerate(eventos) if v.endswith('balon_icono.png')]]

        return {'nombre':nombre,'dorsal':int(dorsal),'etiquetas':etiquetas,'entradas':entradas,'salidas':salidas,'goles':goles}
    def parseEntrenador(entrenadorElem):
        etiquetas_entrenador = [etiqueta.text for etiqueta in entrenadorElem.select('span.span_etiqueta_alin')]
        entrenadorElem = entrenadorElem.text.split(' ')

        nombre = ' '.join(entrenadorElem)
        for etiqueta in etiquetas_entrenador:
            nombre = nombre.replace(etiqueta,'')
        return {'nombre': nombre.strip()}
    


    equipo_local = soup.select_one('div.equipo:nth-child(1) > a:nth-child(2) > span:nth-child(1)').text.replace('d&#39;','')
    equipo_visitante = soup.select_one('div.equipo:nth-child(3) > a:nth-child(2) > span:nth-child(1)').text

    titulares_local = [parseJugador(titular) for titular in local_titulares]
    suplentes_local = [parseJugador(titular) for titular in local_suplentes]
    jugadores_local = titulares_local + suplentes_local
    tecnico_local = [parseEntrenador(tecnico) for tecnico in local_tecnicos]

    titulares_visitante = [parseJugador(titular) for titular in visitante_titulares]
    suplentes_visitante = [parseJugador(titular) for titular in visitante_suplentes]
    jugadores_visitante= titulares_visitante + suplentes_visitante
    tecnicos_visitante = [parseEntrenador(tecnico) for tecnico in visitante_tecnicos]

    
 #Calculamos los goles que ha metido el equipo local en todo el partido

    goles = 0 #decimos que goles tiene una cantidad de 0 para ir añadiendo
    for jugadorElem in local_titulares + local_suplentes: #le decimos que de los jugadores del equipo local que hay en jugadorElemen
        jugador = parseJugador(jugadorElem) #Igualamos la variable jugador al pareseo creado anteriormente
        goles += len(jugador['goles']) #Añadimos la cantidad de goles que encuentr en jugador a los goles
    goles_local_total = goles #Creamos la variable goles_local_total y la igualamos con goles. 
    #La variable goles podría llamarse desde el principio goles_local_total pero de esta forma las siguientes tienen la misma forma.

 #Calculamos los goles que ha metido el equipo visitante en todo el partido       
    goles = 0
    for jugadorElem in visitante_titulares + visitante_suplentes:
        jugador = parseJugador(jugadorElem)
        goles += len(jugador['goles'])
    goles_visitante_total = goles

 #Calculamos los goles que ha metido el equipo local en la primera parte (cada parte dura 45 minutos)   
    goles = 0
    for jugadorElem in local_titulares + local_suplentes:
        jugador = parseJugador(jugadorElem)
        for minuto in jugador['goles']:
            if minuto < 45:
                goles += 1
    goles_primera_local = goles

 #Calculamos los goles que ha metido el equipo local en la segunda parte
    goles = 0
    for jugadorElem in local_titulares + local_suplentes:
        jugador = parseJugador(jugadorElem)
        for minuto in jugador['goles']:
            if minuto > 45:
                goles += 1
    goles_segunda_local = goles

 #Calculamos los goles que ha metido el equipo visitante en la primera parte
    goles = 0
    for jugadorElem in visitante_titulares + visitante_suplentes:
        jugador = parseJugador(jugadorElem)
        for minuto in jugador['goles']:
            if minuto < 45:
                goles += 1
    goles_primera_visitante = goles

 #Calculamos los goles que ha metido el equipo visitante en la segunda parte  
    goles = 0
    for jugadorElem in visitante_titulares + visitante_suplentes:
        jugador = parseJugador(jugadorElem)
        for minuto in jugador['goles']:
            if minuto > 45:
                goles += 1
    goles_segunda_visitante = goles

 #Calculamos los goleadores de ambos equipos    
    goleadores = [] #creamos una lista vacía
    for jugadorElem in local_titulares + local_suplentes + visitante_titulares + visitante_suplentes: #seleccionamos todos los jugadores
        jugador = parseJugador(jugadorElem)
        if len(jugador['goles']) > 0: # Si un jugador tiene más de 0 goles se añade a goleadores
            goleadores.append(jugador)

 #Calculamos los goleadores del equipo local    
    goleadores_local = []
    for jugadorElem in local_titulares + local_suplentes:
        jugador = parseJugador(jugadorElem)
        if len(jugador['goles']) > 0:
            goleadores_local.append(jugador)

 #Calculamos los goleadores del equipo visitante     
    goleadores_visitante = []
    for jugadorElem in visitante_titulares + visitante_suplentes:
        jugador = parseJugador(jugadorElem)
        if len(jugador['goles']) > 0:
            goleadores_visitante.append(jugador)

 #Calculamos los goleadores del equipo local en la primera parte             
    goleadores_local_primera = []
    for jugadorElem in local_titulares + local_suplentes:
        jugador = parseJugador(jugadorElem)
        for minuto in jugador['goles']:
            if minuto <45:
                goleadores_local_primera.append(jugador)
                
 #Calculamos los goleadores del equipo local en la segunda parte                
    goleadores_local_segunda = []
    for jugadorElem in local_titulares + local_suplentes:
        jugador = parseJugador(jugadorElem)
        for minuto in jugador['goles']:
            if minuto >45:
                goleadores_local_segunda.append(jugador)
                
#Calculamos los goleadores del equipo visitante en la primera parte    
    goleadores_visitante_primera = []
    for jugadorElem in visitante_titulares + visitante_suplentes:
        jugador = parseJugador(jugadorElem)
        for minuto in jugador['goles']:
            if minuto <45:
                goleadores_visitante_primera.append(jugador)
                
#Calculamos los goleadores del equipo visitante en la segunda parte 
    goleadores_visitante_segunda = []
    for jugadorElem in visitante_titulares + visitante_suplentes:
        jugador = parseJugador(jugadorElem)
        for minuto in jugador['goles']:
            if minuto >45:
                goleadores_visitante_segunda.append(jugador)

#Calculamos la cantidad de jugadores que entran al campo del equipo local                 
    entradas = 0
    for jugadorElem in local_suplentes:
        jugador = parseJugador(jugadorElem)
        entradas += len(jugador['entradas'])
    local_entradas = entradas
    
#Calculamos la cantidad de jugadores que entran al campo del equipo visitante 
    entradas = 0
    for jugadorElem in visitante_suplentes:
        jugador = parseJugador(jugadorElem)
        entradas += len(jugador['entradas'])
    visitante_entradas = entradas
    
#Calculamos los jugadores que entran al campo deambos equipos
    entran = []
    for jugadorElem in local_suplentes + visitante_suplentes:
        jugador = parseJugador(jugadorElem)
        if len(jugador['entradas']) > 0:
            entran.append(jugador)
            
#Calculamos los jugadores que entran al campo del equipo local
    entran_local = []
    for jugadorElem in visitante_suplentes:
        jugador = parseJugador(jugadorElem)
        if len(jugador['entradas']) > 0:
            entran_local.append(jugador)
            
#Calculamos los jugadores que entran al campo del equipo visitante
    entran_visitante = []
    for jugadorElem in visitante_suplentes:
        jugador = parseJugador(jugadorElem)
        if len(jugador['entradas']) > 0:
            entran_visitante.append(jugador)
            
#Calculamos los jugadores que salen del campo de ambos equipos
    salen = []
    for jugadorElem in local_titulares + visitante_titulares:
        jugador = parseJugador(jugadorElem)
        if len(jugador['salidas']) > 0:
            salen.append(jugador)
            
#Calculamos los jugadores que salen del campo del equipo local
    salen_local = []
    for jugadorElem in local_titulares:
        jugador = parseJugador(jugadorElem)
        if len(jugador['salidas']) > 0:
            salen_local.append(jugador)
            
#Calculamos los jugadores que salen del campo del equipo visitante
    salen_visitante = []
    for jugadorElem in visitante_titulares:
        jugador = parseJugador(jugadorElem)
        if len(jugador['salidas']) > 0:
            salen_visitante.append(jugador) 
            
#Calculamos la fecha formateada. Requerida por el tema de la página de Github para subir los posts.          
    fecha = soup.select_one('.estadio_barra').text.split('|')[-2].strip()
    fechaFormateada = fecha.split('-')
    fechaFormateada = "-".join(reversed(fechaFormateada))
    
    
#Creamos un diccionario con todas las variables extraídas.    
    return {'jornada':soup.select_one('button.btn:nth-child(2)').text.split(' ')[17].rjust(2, '0'), #Separamos el texto en espacios y selecionamos la posición del número de jornada. El rjust es para añadir 0 al final y que todas las jornadas tengan las mismas cifras. Esto es importante a la hora de ordenar los archivos en la página de github.
     'fecha': fechaFormateada,
     'estadio':soup.select_one('.nombre_campo').text.replace('|',''), 
     'equipo_local':soup.select_one('div.equipo:nth-child(1) > a:nth-child(2) > span:nth-child(1)').text.replace('"','').replace('d&#39;',''), #Se remplaza d&#39 ya que hay equipos con este error en su nombre y así se limpia.
     'equipo_visitante':soup.select_one('div.equipo:nth-child(3) > a:nth-child(2) > span:nth-child(1)').text.replace('"','').replace('d&#39;',''),
     'resultado':  str(goles_local_total) + " - " + str(goles_visitante_total),
     'resultado_primera':str(goles_primera_local) + " - " + str(goles_primera_visitante),
     'resultado_segunda': str(goles_segunda_local) + " - " + str(goles_segunda_visitante),
     'equipo_ganador': max({equipo_local: int(goles_local_total), equipo_visitante: int(goles_visitante_total)}),
     'equipo_perdedor':min({equipo_local: int(goles_local_total), equipo_visitante: int(goles_visitante_total)}),
     'equipo_ganador_primera': max({equipo_local: int(goles_primera_local), equipo_visitante: int(goles_primera_visitante)}),
     'equipo_perdedor_primera': min({equipo_local: int(goles_primera_local), equipo_visitante: int(goles_primera_visitante)}),
     'equipo_ganador_segunda': max({equipo_local: int(goles_segunda_local), equipo_visitante: int(goles_segunda_visitante)}),
     'equipo_perdedor_segunda': min({equipo_local: int(goles_segunda_local), equipo_visitante: int(goles_segunda_visitante)}),
     'tecnico_local':', '.join([tecnico['nombre'].strip().title() for tecnico in tecnico_local]), #.Title lo utilizamos para que la primera letra del nombre y apellidos sea en mayuscula
     'titulares_local':', '.join([jugador['nombre'].title() for jugador in titulares_local]),
     'suplentes_local':', '.join([jugador['nombre'].title() for jugador in suplentes_local]),
     'jugadores_local':titulares_local + suplentes_local ,
     'tecnico_visitante':', '.join([tecnico['nombre'].strip().title() for tecnico in tecnicos_visitante ]),
     'titulares_visitante':', '.join([jugador['nombre'].title() for jugador in titulares_visitante]),
     'suplentes_visitante':', '.join([jugador['nombre'].title() for jugador in suplentes_visitante]),
     'jugadores_visitante':titulares_visitante + suplentes_visitante,
     'goles_total': goles_visitante_total + goles_local_total,
     'goles_local_total': goles_local_total,
     'goles_primera_local': goles_primera_local,
     'goles_segunda_local' : goles_segunda_local,
     'goles_visitante_total': goles_visitante_total,
     'goles_primera_visitante': goles_primera_visitante,
     'goles_segunda_visitante': goles_segunda_visitante,
     'goleadores': ', '.join([jugador['nombre'].title() for jugador in goleadores]),
     'goles_ganador_primera': max({goles_primera_local, goles_primera_visitante}),
     'goles_perdedor_primera': min({goles_primera_local, goles_primera_visitante}),
     'goles_ganador_segunda': max({goles_segunda_local, goles_segunda_visitante}),
     'goles_perdedor_segunda': min({goles_segunda_local, goles_segunda_visitante}),
     'goleadores_primera': goleadores_local_primera + goleadores_visitante_primera,
     'goleadores_local': ', '.join([jugador['nombre'].title() for jugador in goleadores_local]),
     'goleadores_local_primera': ', '.join([jugador['nombre'].title() for jugador in goleadores_local_primera]),
     'goleadores_local_segunda': ', '.join([jugador['nombre'].title() for jugador in goleadores_local_segunda]),
     'goleadores_visitante': ', ' .join([jugador['nombre'].title() for jugador in goleadores_visitante]),
     'goleadores_visitante_primera': ', '.join([jugador['nombre'].title() for jugador in goleadores_visitante_primera]),
     'goleadores_visitante_segunda': ', '.join([jugador['nombre'].title() for jugador in goleadores_visitante_segunda]),
     'primer_goleador' : (goleadores_local_primera + goleadores_visitante_primera)[0]['nombre'].title(),
     'primer_gol_minuto': str((goleadores_local_primera + goleadores_visitante_primera)[0]['goles']).replace("[", "").replace("]", "") , 
     'primer_goleador_segunda' : (goleadores_local_segunda + goleadores_visitante_segunda)[0]['nombre'].title(),
     'primer_gol_minuto_segunda': str((goleadores_local_segunda + goleadores_visitante_segunda)[0]['goles']).replace("[", "").replace("]", "")[0:2] ,       
     'entran': ', ' .join([jugador['nombre'].title() for jugador in entran]),
     'salen': ', ' .join([jugador['nombre'].title() for jugador in salen]),
     'entran_local': ', ' .join([jugador['nombre'].title() for jugador in entran_local]),
     'entran_visitante': ', ' .join([jugador['nombre'].title() for jugador in entran_visitante]),
     'salen_local': ', ' .join([jugador['nombre'].title() for jugador in salen_local]),
     'salen_visitante': ', ' .join([jugador['nombre'].title() for jugador in salen_visitante]),
     
    }


### Almacenamos el diccionario anterior en una variable

Creamos una variable con todos los datos del diccionario anterior y lo probamos con una url para comprobar su funcionamiento.

In [465]:
partido = parsearPartido('https://resultadosffcv.isquad.es/partido.php?id_partido=3679064&id_torneo=39882&jornada=3')
partido

IndexError: list index out of range

### Creación de las partes de texto de la crónica

Los textos de la crónica se crearán juntando 5 bloques diferentes:

#### Primer bloque
La cabecera. 

Necesaria para el tema utilizado para la página de github. En este se le dice el tipo de página que es y la marcamos como post. Después le asignamos un nombre que será el que se mostrará en el índice de la página principal y le asignamos primero el número de la jornada para que se ordenen de forma descendente y así se muestren primero las últimas disputadas. A continuación, se le asigna el nombre del equipo local y el del visitante para saber los equipos del partido.

In [8]:
layout = [ '--- \n\
layout: post \n\
title: "Jornada {jornada}: {equipo_local} VS {equipo_visitante}"\n\
---']
    
    
def head(partido):
    
    cronica = random.choice(layout).format(**partido)
    return cronica

#### Segundo bloque
La entradilla.

Formada por dos partes: la presentación, que contiene el número de la jornada, los equipos y el estadio y las alineaciones que se exponen los jugadores titulares de cada equipo y sus entrenadores.
De cada una de estas partes se han creado 5 posibles opciones que se asignarán de forma aleatoria a través de random.coice y los datos se obtienen del diccionario creado anteriormente con .format(**partido). 

In [475]:
def cronicaEntradilla(partido):
    presentacion_entradilla = \
    [
    "El partido de la jornada {jornada} ha enfrentado a {equipo_local} contra el {equipo_visitante} en el mítico campo {estadio}. ",
    "El {estadio} albergó el partido correspondiente a la jornada {jornada} entre {equipo_local} y {equipo_visitante}. ",
    "El {equipo_visitante} visitó el {estadio}  ,estadio del {equipo_local}, para enfrentarse en la jornada {jornada}. ",
    "El {equipo_local} y el {equipo_visitante} jugaron el partido de la jornada {jornada} en el {estadio}. ",
    "El {equipo_ganador} sumó tres puntos a su casillero tras ganar en la jornada {jornada} por {resultado} contra el {equipo_perdedor} en {estadio}. ",
    "El {equipo_local} jugó en su mítico estadio, el {estadio}, el partido de la jornada {jornada} contra el {equipo_visitante}. "
    ]
    alineaciones_entradilla = \
    [
    "El equipo local salió con {titulares_local} a decisión de su cuerpo técnico {tecnico_local}. Por su parte, los entrenadores del equipo local, dirigido por {tecnico_visitante} salió al campo con {titulares_visitante}. ",
    "El entrenador local , {tecnico_local} , alineó a {titulares_local} mientras que el cuerpo técnico encabezado por {tecnico_visitante[0]} decidió salir con {titulares_visitante}. ",
    "{titulares_local}, fueron los elegidos por el entrenador local, {tecnico_local}, para salir de inicio. Los visitantes, dirigidos por {tecnico_visitante} alineó a {titulares_visitante}. ",
    "Por parte de los locales, el cuerpo técnico de {equipo_local} , formado por {tecnico_local} alineó a: {titulares_local} mientras que los visitantes saltaron al campo con {titulares_visitante} a decisión de {tecnico_visitante}. ",
    "En esta ocasión el equipo entrenado por {tecnico_local} jugó de inicio con {titulares_local}, mientras que por parte del equipo visitante salieron {titulares_visitante} a decisión de {tecnico_visitante}. "
    ]
    
    cronica = random.choice(presentacion_entradilla).format(**partido) + \
    random.choice(alineaciones_entradilla).format(**partido)
    return cronica

#### Tercer bloque
La primera parte.


En esta dividimos diferentes textos según si hay empate con goles, empate sin goles o si no hay empate. De cada una de estas tres opciones se han creado textos para el resultado y los goles.
Posteriormente, se identifica las situaciones del partido detalladas anteriormente (empate con goles, empate sin goles o si no hay empate) y se añaden variable que muestran si han habido sitaciones excepcionales como muchos goles, un ganador claro...

Por último se genera una cadena de texto vacía llamada cronica en la que se añadirán las frases según los textos escritos anteriormente según las situaciones del partido y al final se le añaden las situaciones excepcionales.

In [467]:
def cronicaPrimeraParte(partido):
    
    resultado_primera_emp_goles = [ \
    "El primer tiempo acabó con un empate a {goles_primera_local}. ", 
    "El marcador mostraba un empate a {goles_primera_local} al finalizar los primeros 45 minutos reglamentarios. ",
    "La igualdad reinó durante los primeros 45 minutos y ambos equipos se fueron a vestuarios habiendo marcado {goles_primera_local} goles.  ",
    "Los dos equipos tuvieron oportunidad de irse a vestuarios dominando el marcador, pero al acabar la primera parte el marcador mostraba un empate a {goles_primera_local}. ",
    "La primera parte fue muy disputada y ninguno de los dos equipos se fue dominando y el marcado mostraba el {resultado_primera}.  "                         
                                ]

    goles_primera_emp_goles = [\
    "Los goles llegaron de la mano de {goleadores_local_primera} para los locales y de {'goleadores_visitante_primera'} para el {equipo_visitante}. ",
    "{goleadores_local_primera} por parte de {equipo_local} mientras que los visitante aprovecharon la puntería de {goleadores_visitante_primera}. ",
    "El partido empezó con el gol de {primer_goleador} en el minuto {primer_gol_minuto} pero no supieron mantener la ventaja. ",
    "Ningún entrenador se fue satisfecho al vestuario ya que ninguno de los dos equipos supo imponer su ley pese a los goles locales de {goleadores_local_primera}  y {goleadores_visitante_primera} por parte del {equipo_visitante}. " ,                 
    "El primer gol de {primer_goleador} al minuto {primer_gol_minuto} no fue suficiente para poner a su equipo por delante al descanso. ",
                           
                                ]

    resultado_primera_emp_nogoles = [\
     "Primera parte aburrida sin goles en el marcador. ",
     "Los aficionados no vivieron el partido más interesante del campeonato y el 0-0 reinaba en el marcador al final de los primeros 45 minutos. ",
     "Sosa primera parte en la que ningún equipo pudo celebrar ningún gol. ",
     "Durante el primer tiempo del partido ninguno de los jugadores consiguió marcar gol, por lo que durante los 45 primeros minutos el marcador no se movió del 0-0 inicial. ",
     "En la primera parte ninguno de los equipos estuvo acertado de cara al gol, por lo que los primeros 45 minutos concluyeron con el mismo resultado de 0-0. "                            
                                ]     


    resultado_noemp_goles = [\
     
     "El {equipo_ganador_primera} se impuso en los primeros 45 minutos por {resultado_primera}. " ,
     "El {equipo_ganador_primera} se fue con los deberes hechos al descanso con un {resultado_primera} a su favor. ",
     "A final de los primeros 45 minutos reglamentarios el marcador mostraba {resultado_primera} a favor del {equipo_ganador_primera}. ",
     "La primera mitad acabó con un {resultado_primera} favorable a {equipo_ganador_primera}. " ,
     "Buena primera parte del {equipo_ganador_primera} que se fue al vestuario con un {resultado_primera} a su favor. "     
                                ]

    goles_noemp_goles = [\
     "La puntería de {goleadores_local_primera} {goleadores_visitante_primera}  fue la culpable de que el marcador se moviera del 0-0. ",
     "El partido empezó bien para {primer_goleador} que fue el encargado de abrir el marcador en el minuto {primer_gol_minuto}. ", 
     "El primer gol del partido lo metió {primer_goleador} en el minuto {primer_gol_minuto}. "
                                ] 
    
    parte1_emp_goles = [
        random.choice(resultado_primera_emp_goles) + \
        random.choice(goles_primera_emp_goles)
    ]
    parte1_emp_nogoles= [
         random.choice(resultado_primera_emp_nogoles)
    ]
    parte1_noemp_goles =[
       random.choice(resultado_noemp_goles)+ \
       random.choice(goles_noemp_goles)
    ]
    #identificar la situación del partido
    empate = partido['goles_primera_local'] == partido['goles_primera_visitante']
    hay_goles = partido['goles_primera_local'] + partido['goles_primera_visitante'] > 0
    hay_muchos_goles = partido['goles_primera_local'] + partido['goles_primera_visitante'] > 3
    hay_un_claro_ganador = abs(partido['goles_primera_local'] - partido['goles_primera_visitante']) > 2 
    
    
    cronica = ""
    plantilla_cronica = None
    if empate and hay_goles:
        plantilla_cronica = parte1_emp_goles
    elif empate and not hay_goles:
        plantilla_cronica = parte1_emp_nogoles
    elif not empate and hay_goles:
        plantilla_cronica = parte1_noemp_goles
    
    cronica += random.choice(plantilla_cronica).format(**partido)
     
    if hay_muchos_goles:
        cronica += "La primera parte fue un festival de goles. "
    if hay_un_claro_ganador:
        cronica += "Está claro que el {equipo_ganador_primera} salió más concentrado al partido. ".format(**partido)

        
    return cronica

#### Cuarto bloque
La segunda parte.

Muy similar a la primera parte, pero las situaciones se identifican a través del resultado del partido y no solo de la segunda parte. 



In [468]:
def cronicaSegundaParte(partido):
    
    resultado_segunda_emp_goles = [ \
    "Un segundo tiempo igualado en el que ambos equipos han metido los mismos goles {goles_segunda_local}. ", 
    "El marcador mostraba un {resultado} al finalizar los 90 minutos reglamentarios al meter ambos equipos {goles_segunda_local} goles. ",
    "La igualdad reinó durante los segundos 45 minutos y ambos equipos marcaron {goles_segunda_local} goles.  ",
    "Los dos equipos tuvieron oportunidad de dominar, pero marcaron ambos {goles_primera_local} goles en la segunda parte y el partido acabó con un {resultado}. ",
    "La segunda parte fue muy disputada y ninguno de los dos equipos consiguió marcar más goles que el rival y el partido acabó en {resultado}.  "                         
    ]

    goles_segunda_emp_goles = [\
        "Los goles llegaron de la mano de {goleadores_local_segunda} para los locales y de {'goleadores_visitante_segunda'} para el {equipo_visitante}. ",
        "{goleadores_local_segunda} por parte de {equipo_local} mientras que los visitante aprovecharon la puntería de {goleadores_visitante_segunda}. ",
        "La segunda parte empezó con el gol de {primer_goleador_segunda} en el minuto {primer_gol_minuto_segunda} pero el rival no bajó los brazós y logró empatar el resultado de la segunda parte dejando un {resultado} en el marcador final. ",
        "Ningún entrenador se fue satisfecho con el trabajo de la segunda parte ya que ninguno de los dos equipos supo imponer su ley pese a los goles locales de {goleadores_local_segunda}  y {goleadores_visitante_segunda} por parte del {equipo_visitante}. " ,                 
        "El gol de {primer_goleador_segunda} al minuto {primer_gol_minuto_segunda} no fue el único de la segunda parte y el partido acabó con un {resultado}. ",

                    ]

    resultado_segunda_emp_nogoles = [\
         "La segunda parte fue muy aburrida, sin goles en el marcador. ",
         "Los aficionados no vivieron el segundo tiempo más interesante del campeonato y el 0-0 reinaba en el marcador al final de los primeros 45 minutos. ",
         "Sosa segunda parte en la que ningún equipo pudo celebrar un gol. ",
         "Durante el segundo tiempo del partido ninguno de los jugadores consiguió marcar gol, por lo que durante los segundos 45 minutos el marcador no se movió del {resultado_primera} inicial. ",
         "En la segunda parte ninguno de los equipos estuvo acertado de cara al gol, por lo que los segundos 45 minutos concluyeron con el mismo resultado de {resultado_primera}. "                            
                                    ]     


    resultado_noemp_goles2 = [\

         "El {equipo_ganador} se impuso al final de los segundos 45 minutos por {resultado}. " ,
         "El {equipo_ganador} se fue con los deberes hechos al final con un {resultado} a su favor. ",
         "A final del tiempo reglamentarios el marcador mostraba {resultado} a favor del {equipo_ganador}. ",
         "La segunda mitad acabó con un {resultado} favorable a {equipo_ganador}. " ,
         "Buena segunda parte del {equipo_ganador} que se fue al vestuario con un {resultado} a su favor. "     
                             ]
    goles_noemp_goles2 = [\
     "La puntería de {goleadores_local_segunda} {goleadores_visitante_segunda}  fue la culpable de que el marcador se moviera del {resultado_primera} del descanso. ",
     "El segundo tiempo empezó bien para {primer_goleador_segunda} que fue el encargado de abrir el marcador de la segunda parte en el minuto {primer_gol_minuto_segunda}. ", 
     "El primer gol de la segunda parte lo metió {primer_goleador_segunda} en el minuto {primer_gol_minuto_segunda}. "
                        ]
    
    
    parte2_emp_goles = [
        random.choice(resultado_segunda_emp_goles) + \
        random.choice(goles_segunda_emp_goles)
    ]
    parte2_emp_nogoles= [
         random.choice(resultado_segunda_emp_nogoles)
    ]
    parte2_noemp_goles =[
       random.choice(resultado_noemp_goles2)+ \
       random.choice(goles_noemp_goles2)
    ]
    #identificar la situación del partido
    empate = partido['goles_segunda_local'] == partido['goles_segunda_visitante']
    hay_goles = partido['goles_segunda_local'] + partido['goles_segunda_visitante'] > 0
    hay_muchos_goles = partido['goles_segunda_local'] + partido['goles_segunda_visitante'] > 3
    hay_un_claro_ganador = abs(partido['goles_segunda_local'] - partido['goles_segunda_visitante']) > 2 
    empate_cero = partido['goles_local_total'] + partido['goles_visitante_total'] == 0

    
    cronica = ""
    plantilla_cronica = None
    if empate and hay_goles:
        plantilla_cronica = parte2_emp_goles
    elif empate and not hay_goles:
        plantilla_cronica = parte2_emp_nogoles
    elif not empate and hay_goles:
        plantilla_cronica = parte2_noemp_goles

    
    
    cronica += random.choice(plantilla_cronica).format(**partido)
     
    if hay_muchos_goles:
        cronica += "La segunda parte fue realmente entretenida. "
    if hay_un_claro_ganador:
        cronica += "Victoria clara del {equipo_ganador} que buscará repetir el resultado en el próximo partido. Todo lo contrario del {equipo_perdedor} que tendrá que corregir los errores si quiere ganar la semana que viene. ".format(**partido)
    if empate_cero:
        cronica += "Esperemos que se hayan guardado los goles para el siguiente partido. "
        
    return cronica

#### Quinto bloque
La ficha técnica

En esta parte final de la crónica añadimos los datos del partido como el nombre de los equipos, jugadores y entrenadores... de forma simple.

Para crear esta ficha se siguen los mismos pasos utilizados en el head, ubicado en el primer bloque. 

In [469]:
def fichaTecnica(partido):

    ficha_tecnica =[ 'Ficha técnica: \n\
    \n\
    {equipo_local}: Titulares: {titulares_local} \n\
    Suplentes: {suplentes_local} \n\
    Cuerpo técnico {tecnico_local} \n\
    \n\
    {equipo_visitante}: Titulares {titulares_visitante}\n\
    Suplentes: {suplentes_visitante} \n\
    Cuerpo técnico {tecnico_visitante} Estadio: {estadio}. Goles: {goleadores}.  ']
    
    cronica = random.choice(ficha_tecnica).format(**partido)
    return cronica


### Extracción de las url de todos los partidos

En la siguiente celda se obtienen las url de todos los partidos para poder crear crónicas de cada partido.

Se generan las variables url, headfers, y soup de la misma forma que al principio del cuaderno con una url de la federación con los partidos según las jornadas.

Otra variable de url es jornadas. Al haber un desplegable con las url de cada jornada se extrae mediante request y BeautifulSoup como anteriormente y se consiguen los 30 links (hay 30 jornadas en la competición).


Otra manera de hacerlo sería eliminar de la url el número de la jornada y remplazarla por llaves vacías ( {} ). Después le decimos que añada números del 1 al 30 al tener 30 jornadas la competición. De esta forma ya tenemos los 30 links de las jornadas, ya que las url tienen el mismo formato. Esta forma se ha dejado comentada en la celda, ya que en esta ocasión se podría hacer, pero no se sabe si en otras páginas se tendría esa facilidad de cambiar.


Después se crea una lista vacía llamada urlJornadas en la que se añaden las url conseguidas y se modifican para conseguir la url completa, ya que en las almacenadas en "jornadas" no se tiene la parte del dominio por lo que no está el link completo.
Para conseguir el link completo se cambia, para cada jornada dentro de jornadas, la url por el nombre del dominio + la url y se añade en la lista vacía creada.

Con esto ya tenemos las url de las jornadas conseguidas y el siguiente paso es extraer las url de todos los partidos disputados en cada jornada. Para ello creamos otra lista vacía llamada "todoslosPartidos" en la que le pedimos que para cada jornada en las url extraídas en le paso anterior obtenga (mediante request y BeautifulSoup) la url de partidos de cada jornada y después añadimos las url de cada jornada a "todoslosPartidos".

Por último, añadimos la parte del dominio para conseguir el link completo y ya tenemos las url de todos los partidos disputados a lo largo de la competición.






In [470]:
url = 'https://resultadosffcv.isquad.es/resultados_partidos.php?id_torneo=39882&jornada=2&id_temp=2122&id_modalidad=1&id_competicion=10558'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
r = requests.get(url,headers=headers)
soup = BeautifulSoup(r.text,'html.parser')

# urlJornada = 'https://resultadosffcv.isquad.es/resultados_partidos.php?id_torneo=39882&jornada={}&id_temp=2122&id_modalidad=1&id_competicion=10558'
# [urlJornada.format(jornada) for jornada in range(1,30)]

jornadas = list(set([ a["href"] for a in soup.select(".capa_jornada >a ")]))
urlJornadas = []

for jornada in jornadas:
    jornada = jornada.replace(jornada, 'https://resultadosffcv.isquad.es/'+jornada)
    urlJornadas.append(jornada)
todoslosPartidos = []
for jornada in urlJornadas:

    r = requests.get(jornada,headers=headers)
    soup = BeautifulSoup(r.text,'html.parser')
    urlpartidosJornada = list(set([ a["href"] for a in soup.select(".td_nombre_resultados > a")]))
    todoslosPartidos = todoslosPartidos + urlpartidosJornada
    
urlPartidos =['https://resultadosffcv.isquad.es/'+partido.replace('partido_estadisticas.php','partido.php') for partido in todoslosPartidos]
urlPartidos


['https://resultadosffcv.isquad.es/partido_estadisticas.php?id_partido=3679195&id_torneo=39882&jornada=20',
 'https://resultadosffcv.isquad.es/partido_estadisticas.php?id_partido=3679194&id_torneo=39882&jornada=20',
 'https://resultadosffcv.isquad.es/partido_estadisticas.php?id_partido=3679201&id_torneo=39882&jornada=20',
 'https://resultadosffcv.isquad.es/partido_estadisticas.php?id_partido=3679196&id_torneo=39882&jornada=20',
 'https://resultadosffcv.isquad.es/partido_estadisticas.php?id_partido=3679200&id_torneo=39882&jornada=20',
 'https://resultadosffcv.isquad.es/partido_estadisticas.php?id_partido=3679197&id_torneo=39882&jornada=20',
 'https://resultadosffcv.isquad.es/partido_estadisticas.php?id_partido=3679198&id_torneo=39882&jornada=20',
 'https://resultadosffcv.isquad.es/partido_estadisticas.php?id_partido=3679199&id_torneo=39882&jornada=20',
 'https://resultadosffcv.isquad.es/partido_estadisticas.php?id_partido=3679064&id_torneo=39882&jornada=3',
 'https://resultadosffcv.isqu

In [473]:
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679201&id_torneo=39882&jornada=20')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679196&id_torneo=39882&jornada=20')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679200&id_torneo=39882&jornada=20')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679198&id_torneo=39882&jornada=20')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679199&id_torneo=39882&jornada=20')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679062&id_torneo=39882&jornada=3')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679102&id_torneo=39882&jornada=8')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679278&id_torneo=39882&jornada=30')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679228&id_torneo=39882&jornada=24')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679207&id_torneo=39882&jornada=21')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679204&id_torneo=39882&jornada=21')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679202&id_torneo=39882&jornada=21')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679205&id_torneo=39882&jornada=21')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679238&id_torneo=39882&jornada=25')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679240&id_torneo=39882&jornada=25')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679236&id_torneo=39882&jornada=25')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679117&id_torneo=39882&jornada=10')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679272&id_torneo=39882&jornada=29')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679177&id_torneo=39882&jornada=17')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679158&id_torneo=39882&jornada=15')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679159&id_torneo=39882&jornada=15')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679045&id_torneo=39882&jornada=1')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679047&id_torneo=39882&jornada=1')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679138&id_torneo=39882&jornada=13')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3825707&id_torneo=39882&jornada=14')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679252&id_torneo=39882&jornada=27')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679250&id_torneo=39882&jornada=27')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679254&id_torneo=39882&jornada=27')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679167&id_torneo=39882&jornada=16')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679168&id_torneo=39882&jornada=16')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679169&id_torneo=39882&jornada=16')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679264&id_torneo=39882&jornada=28')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679263&id_torneo=39882&jornada=28')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679078&id_torneo=39882&jornada=5')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679074&id_torneo=39882&jornada=5')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679079&id_torneo=39882&jornada=5')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679077&id_torneo=39882&jornada=5')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679080&id_torneo=39882&jornada=5')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679075&id_torneo=39882&jornada=5')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679092&id_torneo=39882&jornada=7')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679091&id_torneo=39882&jornada=7')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679093&id_torneo=39882&jornada=7')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679110&id_torneo=39882&jornada=9')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679112&id_torneo=39882&jornada=9')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679111&id_torneo=39882&jornada=9')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3812372&id_torneo=39882&jornada=9')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679192&id_torneo=39882&jornada=19')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679186&id_torneo=39882&jornada=19')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679084&id_torneo=39882&jornada=6')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679219&id_torneo=39882&jornada=23')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679225&id_torneo=39882&jornada=23')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679242&id_torneo=39882&jornada=26')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679247&id_torneo=39882&jornada=26')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679245&id_torneo=39882&jornada=26')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679132&id_torneo=39882&jornada=12')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679135&id_torneo=39882&jornada=12')
urlPartidos.remove('https://resultadosffcv.isquad.es/partido.php?id_partido=3679131&id_torneo=39882&jornada=12')

### Almacenamiento de las crónicas en archivos Markdown

Necesitamos que los archivos sean en formato .md para poder subirlos al tema Jekyll de la página de Github (minima). 

Para ello le pasamos para cada url la variable "urlPartidos" y le decimos que partido es el parseo de cada url y que crónica son todos los bloques creados anteriormente. También se añade el título que tendrá la crónica asignandole ## para formatearlo en Markdown y el nombre del equipo local, sus goles, los goles del equipo visitante y el equipo visitante.

Después le asignamos al *file* ('f') un nombre que comienza con una fecha (requerido por el tema), el número de la jornada y los equipos. Para asignarle el formato ponemos al final del nombre del archivo .md. Por último lo formateamos con los datos del diccionario del partido para que coja los datos y pueda escribir el nombre del archivo con ellos.
Las crónicas las almacenamos en la carpeta "post" que se encuentra dentro del repositorio de github con el tema.

Después cada archivo (f) le escribimos la crónica y lo cerramos. El progreso de estos pasos lo podremos observar gracias tqdm.

In [476]:
for num,url in enumerate(tqdm(losPartidos)):
    partido = parsearPartido(url)
    cronica = head(partido) + '\n\n' +  '## ' + partido['equipo_local'] + ' ' + partido['resultado'] + ' ' + partido['equipo_visitante'] + '\n\n' + cronicaEntradilla(partido) + '\n\n' + cronicaPrimeraParte(partido)  + '\n\n' +  cronicaSegundaParte(partido) + '\n\n' + fichaTecnica(partido)
    f = open('C:/Users/usuario/cronicasfutbol/prueba/{fecha}-j{jornada}_{equipo_local}_vs_{equipo_visitante}.md'.format(**partido).replace('"', ""),'w',encoding="utf-8")
    f.write(cronica)
    f.close()

  2%|█▍                                                                                | 3/172 [00:01<01:32,  1.83it/s]


IndexError: string index out of range